In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#from numpy import genfromtxt
import io
import math
import random
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [32]:
data = pd.read_excel('/content/drive/MyDrive/NNFL Assignment/data.xlsx');
print(data.shape)

xmin = np.min(data, axis = 0)
xmax = np.max(data, axis = 0)
data = (data- xmin)/(xmax-xmin)

data = data.to_numpy();
print(data.shape)


(3411, 61)
(3411, 61)


In [ ]:
m=data.shape[0]
Y=data[:, 60] # class label
X=data[:, 0:60] # feature matrix

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.3, shuffle = True, random_state = 8)

In [ ]:

print(len(X_train),len(X_test))

2387 1024


In [ ]:
def max_lik(x_ts,x,y):
  x1 = np.array([x[i] for (i, val) in enumerate(y) if val == 1. ])
  x2 = np.array([x[i] for (i, val) in enumerate(y) if val == 2. ])
  x3 = np.array([x[i] for (i, val) in enumerate(y) if val == 3. ])
  x4 = np.array([x[i] for (i, val) in enumerate(y) if val == 4. ])
  # evidence P(x)
  e1, e2, e3 , e4 = len(x1)/(len(x)), len(x2)/(len(x)), len(x3)/(len(x)), len(x4)/(len(x))
  m1 = mean(x1)
  m2 = mean(x2)
  m3 = mean(x3)
  m4 = mean(x4)
  cov1 = np.cov(np.transpose(x1))
  cov2 = np.cov(np.transpose(x2))
  cov3 = np.cov(np.transpose(x3))
  cov4 = np.cov(np.transpose(x4))
  l1 = likelihood(x_ts,m1,cov1)
  l2 = likelihood(x_ts,m2,cov2)
  l3 = likelihood(x_ts,m3,cov3)
  l4 = likelihood(x_ts,m3,cov4)
  if max(l1,l2,l3,l4) == l1:
    return 1.
  elif max(l1,l2,l3) == l2:
    return 2.
  elif max(l1,l2,l3) == l3 :
    return 3.
  else:
    return 4.

In [ ]:
def mean(array):
  m = []
  for i in range(60):
    m.append(sum(col(array,i))/len(col(array,i)))
  return m

In [ ]:
def col(array, i):
  return [row[i] for row in array]

In [ ]:
def likelihood(X,u,sig):
  coeff = 1 /((( 2 * 3.14 )** 3.5 )*np.linalg.det(sig)** 0.5 )
  dev = np.array(X)-np.array(u)
  #lik = coeff*np.exp(- 0.5 *np.dot(np.dot(dev,np.linalg.inv(sig)),np.transpose(dev)))
  gauss = -0.5*(np.dot(np.dot(dev,np.linalg.inv(sig)),np.transpose(dev)))
  try:
    lik = coeff*math.exp(gauss)
  except:
    lik = math.inf
  return lik

In [ ]:
def conf_mat(y_pred,y_ts):
  confmat = np.zeros((4,4))
  for i in range(len(y_ts)):
    if y_ts[i] == 1.:
      if y_pred[i] == 1.:
        confmat [0][0] += 1
      if y_pred[i] == 2.:
        confmat [0][1] += 1
      if y_pred[i] == 3.:
        confmat [0][2] += 1
      if y_pred[i] == 4.:
        confmat [0][3] += 1

    if y_ts[i] == 2.:
      if y_pred[i] == 1.:
        confmat [1][0] += 1
      if y_pred[i] == 2.:
        confmat [1][1] += 1
      if y_pred[i] == 3.:
        confmat [1][2] += 1
      if y_pred[i] == 4.:
        confmat [1][3] += 1

    if y_ts[i] == 3.:
      if y_pred[i] == 1.:
        confmat [2][0] += 1
      if y_pred[i] == 2.:
        confmat [2][1] += 1
      if y_pred[i] == 3.:
        confmat [2][2] += 1
      if y_pred[i] == 4.:
        confmat [2][3] += 1

    if y_ts[i] == 4.:
      if y_pred[i] == 1.:
        confmat [3][0] += 1
      if y_pred[i] == 2.:
        confmat [3][1] += 1
      if y_pred[i] == 3.:
        confmat [3][2] += 1
      if y_pred[i] == 4.:
        confmat [3][3] += 1
  return confmat

In [33]:
p_outputs = []
for b in range(len(X_test)):
  pred_op = max_lik(X_test[b],X_train,Y_train)
  p_outputs.append(pred_op)
confmat = conf_mat(p_outputs,Y_test)
print(confmat)
Acc = (confmat[ 0 ][ 0 ] + confmat[ 1 ][ 1 ] + confmat[ 2 ][ 2 ] + confmat[3][3])/sum(sum(confmat))
Acc1 = confmat[ 0 ][ 0 ]/sum(confmat[ 0 ])
Acc2 = confmat[ 1 ][ 1 ]/sum(confmat[ 1 ])
Acc3 = confmat[ 2 ][ 2 ]/sum(confmat[ 2 ])
Acc4 = confmat[ 3 ][ 3 ]/sum(confmat[ 3 ])
print( 'Overall Accuracy : ' + str(Acc*100))
print( 'Accuracy of class 1 : ' + str(Acc1*100))
print( 'Accuracy of class 2 : ' + str(Acc2*100))
print( 'Accuracy of class 3 : ' + str(Acc3*100))
print( 'Accuracy of class 4 : ' + str(Acc4*100))

[[78. 89. 69. 21.]
 [57. 97. 85. 16.]
 [63. 85. 83. 28.]
 [78. 70. 82. 23.]]
Overall Accuracy : 27.44140625
Accuracy of class 1 : 30.35019455252918
Accuracy of class 2 : 38.03921568627451
Accuracy of class 3 : 32.04633204633205
Accuracy of class 4 : 9.090909090909092
